In [35]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import pandas as pd
from Bio import SeqIO
load_dotenv()
nvda = os.getenv("nvda")
client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = nvda
)
from tqdm import tqdm
import numpy as np

In [36]:
fasta_dir = "sequences"

# Mapping of base categories to questions
questions = {
    "protein_coding": "Does this nucleotide sequence encode a protein? Only answer Yes or No. You must start your answer with 'Yes' or 'No'.",
    "enhancer": "Does this nucleotide sequence function as an enhancer in gene regulation? Only answer Yes or No. You must start your answer with 'Yes' or 'No'.",
    "promoter": "Does this nucleotide sequence act as a promoter for transcription initiation? Only answer Yes or No. You must start your answer with 'Yes' or 'No'.",
    "splice_site": "Does this nucleotide sequence contain a splice site for RNA processing? Only answer Yes or No. You must start your answer with 'Yes' or 'No'.",
    "methylated": "Is this nucleotide sequence methylated as part of epigenetic regulation? Only answer Yes or No. You must start your answer with 'Yes' or 'No'."
}

# Dictionary to hold sequence data
sequence_arrays = {}

# Loop through all FASTA files in the directory
for file in os.listdir(fasta_dir):
    if file.endswith(".fasta"):  # Process only FASTA files
        file_path = os.path.join(fasta_dir, file)
        var_name = os.path.splitext(file)[0]  # Use file name (without extension) as key

        # Determine base category
        if var_name.startswith("non_"):
            base_category = var_name[4:]  # Remove "non_" prefix
        else:
            base_category = var_name
        question = questions[base_category]  # Get the corresponding question

        # Read the FASTA file and create a list of dictionaries
        gene_dicts = []
        for record in SeqIO.parse(file_path, "fasta"):
            gene_dict = {
                "file_name": var_name,
                "gene": str(record.seq),
                "question": question,  # Add the question key
            }
            gene_dicts.append(gene_dict)
        
        # Store the list of dictionaries in the dictionary
        sequence_arrays[var_name] = gene_dicts

        print(f"Stored {var_name} with {len(gene_dicts)} genes.")

Stored enhancer with 98 genes.
Stored methylated with 100 genes.
Stored non_enhancer with 98 genes.
Stored non_methylated with 100 genes.
Stored non_promoter with 100 genes.
Stored non_protein_coding with 88 genes.
Stored non_splice_site with 70 genes.
Stored promoter with 100 genes.
Stored protein_coding with 100 genes.
Stored splice_site with 70 genes.


In [37]:
def generate_hallucination(gene_dict, model):
    response = client.chat.completions.create(
            model= model,  #Use the model specified in the constructor

            messages=
            [
            {"role": "system", "content": "You are an expert in genomics."},
            {"role": "user", "content": f"{gene_dict['gene']} Describe the gene in natural language:"}
        ],  #The conversation context to send to the API


            max_tokens=256,  #Maximum length of the response

            temperature=0.6  #Controls randomness/creativity of the response
        )
    return response.choices[0].message.content

In [38]:
def generate_classification(gene_dict, model):
    response = client.chat.completions.create(
            model = model,  #Use the model specified in the constructor

            messages=
            [
            {"role": "system", "content": "You are an expert in genomics."},
            {"role": "user", "content": f"{gene_dict['gene']} {gene_dict['question']}"}
        ],  #The conversation context to send to the API


            max_tokens=256,  #Maximum length of the response

            temperature=0.6  #Controls randomness/creativity of the response
        )
    return response.choices[0].message.content

In [39]:
def generate_hallucination_classification(gene_dict, model):
    response = client.chat.completions.create(
            model = model,  #Use the model specified in the constructor

            messages=
            [
            {"role": "system", "content": "You are an expert in genomics."},
            {"role": "user", "content": f"{gene_dict['gene']} {gene_dict['hallucination']} {gene_dict['question']}"}
        ],  #The conversation context to send to the API


            max_tokens=256,  #Maximum length of the response

            temperature=0.6  #Controls randomness/creativity of the response
        )
    return response.choices[0].message.content

In [40]:
all_results = []

MODELS = [
    "meta/llama3-8b-instruct",
    "meta/llama-3.1-8b-instruct",
    "meta/llama-3.3-70b-instruct",
    "tiiuae/falcon3-7b-instruct",
    "qwen/qwen2.5-7b-instruct",
    ]

for model_idx, model in enumerate(MODELS):
    model_progress = tqdm(total=len(sequence_arrays), desc=f"Model [{model_idx+1}/6]: {model.split('/')[-1]}")
    
    for file_idx, (file, sequences) in enumerate(sequence_arrays.items()):
        file_progress = tqdm(total=len(sequences), desc=f"File [{file_idx+1}/{len(sequence_arrays)}]: {file}", leave=False)
        
        for gene_idx, gene in enumerate(sequences):
            file_progress.set_postfix_str(f"Gene {gene_idx+1}/{len(sequences)}")
            
            result = gene.copy()
            result['model'] = model
            
            try:
                result['hallucination'] = generate_hallucination(result, model)
            except Exception as e:
                print(f"Error generating hallucination for {file}, gene {gene_idx+1}: {str(e)}")
                result['hallucination'] = None
            
            try:
                result['hallucination_classification'] = generate_hallucination_classification(result, model)
            except Exception as e:
                print(f"Error generating hallucination classification for {file}, gene {gene_idx+1}: {str(e)}")
                result['hallucination_classification'] = None
            
            try:
                result['classification'] = generate_classification(result, model)
            except Exception as e:
                print(f"Error generating classification for {file}, gene {gene_idx+1}: {str(e)}")
                result['classification'] = None
            
            all_results.append(result)
            
            file_progress.update(1)
        
        file_progress.close()
        model_progress.update(1)
    
    model_progress.close()



Model [7/6]: gemma-2-27b-it:   0%|          | 0/10 [39:33<?, ?it/s]
                                                                                  
                                                                                        
                                                                                      
                                                                                            
                                                                                          
                                                                                            
                                                                                         
                                                                                      
                                                                                            
                                                                                      
Model [5/6]: qwen2.5-7b-instruct: 100%

In [46]:
df = pd.DataFrame(all_results)

# Add expected correct answer column
df['correct'] = df['file_name'].str.contains("non", case=True, na=False).map({True: "No", False: "Yes"})

# Clean the classification outputs
df['hallucination_classification'] = df['hallucination_classification'].str.replace(".", "", regex=False)
df['classification'] = df['classification'].str.replace(".", "", regex=False)


In [47]:
df['hallucination_classification'] = df['hallucination_classification'].apply(
    lambda x: 'Yes' if isinstance(x, str) and x.split()[0].lower()[:3] == 'yes' else
              'No' if isinstance(x, str) and x.split()[0].lower()[:2] == 'no' else np.nan
)
print(df['hallucination_classification'].values)

df['classification'] = df['classification'].apply(
    lambda x: 'Yes' if isinstance(x, str) and x.split()[0].lower()[:3] == 'yes' else
              'No' if isinstance(x, str) and x.split()[0].lower()[:2] == 'no' else np.nan
)
print(df['classification'].values)

['Yes' 'Yes' 'Yes' ... 'No' 'No' 'No']
['No' 'No' 'No' ... 'No' 'No' 'No']


In [48]:
df[df['hallucination_classification'].isna()]['model'].value_counts()

model
meta/llama-3.1-8b-instruct    4
Name: count, dtype: int64

In [49]:
df[df['classification'].isna()]['model'].value_counts()

Series([], Name: count, dtype: int64)

In [50]:
df = df.dropna()

df['hallucination_correct'] = df['hallucination_classification'] == df['correct']
df['no_hallucination_correct'] = df['classification'] == df['correct']
df.to_csv("trail_1_open.csv", index=False)

In [51]:
enhancer_hallucination_correct = df[df['file_name'].str.contains("enhancer")].groupby('model')['hallucination_correct'].mean()
enhancer_no_hallucination_correct = df[df['file_name'].str.contains("enhancer")].groupby('model')['no_hallucination_correct'].mean()

promoter_hallucination_correct = df[df['file_name'].str.contains("promoter")].groupby('model')['hallucination_correct'].mean()
promoter_no_hallucination_correct = df[df['file_name'].str.contains("promoter")].groupby('model')['no_hallucination_correct'].mean() 

splice_site_hallucination_correct = df[df['file_name'].str.contains("splice_site")].groupby('model')['hallucination_correct'].mean()     
splice_site_no_hallucination_correct = df[df['file_name'].str.contains("splice_site")].groupby('model')['no_hallucination_correct'].mean()

methylated_hallucination_correct = df[df['file_name'].str.contains("methylated")].groupby('model')['hallucination_correct'].mean()   
methylated_no_hallucination_correct = df[df['file_name'].str.contains("methylated")].groupby('model')['no_hallucination_correct'].mean() 

protein_coding_hallucination_correct = df[df['file_name'].str.contains("protein_coding")].groupby('model')['hallucination_correct'].mean()   
protein_coding_no_hallucination_correct = df[df['file_name'].str.contains("protein_coding")].groupby('model')['no_hallucination_correct'].mean()

data = {
    'enhancer_hallucination_correct': enhancer_hallucination_correct,
    'enhancer_no_hallucination_correct': enhancer_no_hallucination_correct,
    'promoter_hallucination_correct': promoter_hallucination_correct,
    'promoter_no_hallucination_correct': promoter_no_hallucination_correct,
    'splice_site_hallucination_correct': splice_site_hallucination_correct,
    'splice_site_no_hallucination_correct': splice_site_no_hallucination_correct,
    'methylated_hallucination_correct': methylated_hallucination_correct,
    'methylated_no_hallucination_correct': methylated_no_hallucination_correct,
    'protein_coding_hallucination_correct': protein_coding_hallucination_correct,
    'protein_coding_no_hallucination_correct': protein_coding_no_hallucination_correct
}

pivot_data = []

for feature in ['enhancer', 'promoter', 'splice_site', 'methylated', 'protein_coding']:
    hall_series = data[f'{feature}_hallucination_correct']

    no_hall_series = data[f'{feature}_no_hallucination_correct']

    for model in hall_series.index:
        pivot_data.append({
            'model': model,
            'hallucination_status': 'Hallucination',
            'feature': feature,
            'correct_count': hall_series[model]
        })

        pivot_data.append({
            'model': model,
            'hallucination_status': 'No Hallucination',
            'feature': feature,
            'correct_count': no_hall_series[model]
        })

long_df = pd.DataFrame(pivot_data)

pivot_df = long_df.pivot_table(
    index=['model', 'hallucination_status'],
    columns='feature',
    values='correct_count'
)

feature_order = ['enhancer', 'promoter', 'splice_site', 'methylated', 'protein_coding']
pivot_df = pivot_df[feature_order]

pivot_df['AVG'] = pivot_df.mean(axis=1)

pivot_df['AVG'] = pivot_df['AVG'].round(4) 

pivot_df



feature                                           enhancer  promoter  \
model                       hallucination_status                       
meta/llama-3.1-8b-instruct  Hallucination              0.5       0.5   
                            No Hallucination           0.5       0.5   
meta/llama-3.3-70b-instruct Hallucination              0.5       0.5   
                            No Hallucination           0.5       0.5   
meta/llama3-8b-instruct     Hallucination              0.5       0.5   
                            No Hallucination           0.5       0.5   
qwen/qwen2.5-7b-instruct    Hallucination              0.5       0.5   
                            No Hallucination           0.5       0.5   
tiiuae/falcon3-7b-instruct  Hallucination              0.5       0.5   
                            No Hallucination           0.5       0.5   

feature                                           splice_site  methylated  \
model                       hallucination_status                            
meta/llama-3.1-8b-instruct  Hallucination            0.500000       0.495   
                            No Hallucination         0.500000       0.500   
meta/llama-3.3-70b-instruct Hallucination            0.485714       0.490   
                            No Hallucination         0.478571       0.500   
meta/llama3-8b-instruct     Hallucination            0.500000       0.500   
                            No Hallucination         0.500000       0.500   
qwen/qwen2.5-7b-instruct    Hallucination            0.500000       0.500   
                            No Hallucination         0.500000       0.500   
tiiuae/falcon3-7b-instruct  Hallucination            0.500000       0.500   
                            No Hallucination         0.500000       0.500   

feature                                           protein_coding     AVG  
model                       hallucination_status                          
meta/llama-3.1-8b-instruct  Hallucination               0.585106  0.5160  
                            No Hallucination            0.398936  0.4798  
meta/llama-3.3-70b-instruct Hallucination               0.430851  0.4813  
                            No Hallucination            0.510638  0.4978  
meta/llama3-8b-instruct     Hallucination               0.563830  0.5128  
                            No Hallucination            0.202128  0.4404  
qwen/qwen2.5-7b-instruct    Hallucination               0.441489  0.4883  
                            No Hallucination            0.468085  0.4936  
tiiuae/falcon3-7b-instruct  Hallucination               0.372340  0.4745  
                            No Hallucination            0.457447  0.4915